### This is an example how to build an index with lancdb and make hybrid search using all-MiniLM-L6-v2 embeddings

In [ ]:
from lancedb.embeddings import get_registry
from lancedb.pydantic import LanceModel, Vector
model = get_registry().get("sentence-transformers").create(name="all-MiniLM-L6-v2", device="cpu")

In [22]:
from datasets import load_dataset
ds_builder = load_dataset("rotten_tomatoes", split='train')
ds_builder = ds_builder.to_pandas()

,text,label
0,the rock is destined to be the 21st century's ...,1
1,"the gorgeously elaborate continuation of "" the...",1
2,effective but too-tepid biopic,1
3,if you sometimes like to go to the movies to h...,1
4,"emerges as something rare , an issue movie tha...",1
...,...,...
8525,any enjoyment will be hinge from a personal th...,0
8526,if legendary shlockmeister ed wood had ever ma...,0
8527,hardly a nuanced portrait of a young woman's b...,0
8528,"interminably bleak , to say nothing of boring .",0


In [23]:
class VanillaDocuments(LanceModel):
    vector: Vector(model.ndims()) = model.VectorField()  # Default field
    text: str = (model.SourceField())  # the Columns (field) in DB whose Embedding we'll create

In [25]:
!rm -rf ./db

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [26]:
import lancedb
db = lancedb.connect("./db")
vanilla_table = db.create_table("vanilla_documents", schema=VanillaDocuments)
vanilla_table.add(ds_builder)  # ingest docs with auto-vectorization
vanilla_table.create_fts_index("text")  

In [20]:
QUERY = "implement corpus management with event handling"
vanilla_table.search(QUERY, query_type="hybrid").limit(3).to_pandas()

,vector,text,_relevance_score
0,"[-0.02525873, -0.039976526, 0.02436699, 0.0064...",corpus collosum -- while undeniably interestin...,0.016393
1,"[0.0061905687, 0.07616369, -0.03621763, -0.031...",a literate presentation that wonderfully weave...,0.016393
2,"[-0.009595144, -0.044449616, 0.023243582, -0.0...","it's traditional moviemaking all the way , but...",0.016129
